#### DataSet Downloads

https://www.kaggle.com/datasets/rajathmc/cornell-moviedialog-corpus

#### Arquivos Usados
* movie_lines.txt
* movie_conversations.txt

In [1]:
import numpy as np
import tensorflow as tf
import re
import time
import pandas as pd
from deep_translator import GoogleTranslator

2023-06-23 22:45:34.973590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 22:45:35.013880: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 22:45:35.014827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 22:45:35.730862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Pré Processamento da Base

In [2]:
path_root = '/opt/dna/chat_bot/datalake/transient/'
filename = 'movie_lines.txt'
linhas = open(path_root + filename, encoding='utf-8').read().split('\n')

In [3]:
linhas[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [4]:
len(linhas)

304714

In [5]:
path_root = '/opt/dna/chat_bot/datalake/transient/'
filename = 'movie_conversations.txt'
conversas = open(path_root + filename, encoding='utf-8').read().split('\n')

In [6]:
conversas[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [7]:
len(conversas)

83098

In [8]:
id_para_linha = {}
for linha in linhas:
    _linha = linha.split(' +++$+++ ')
    if len(_linha) == 5:
        id_para_linha[_linha[0]] = _linha[4]

In [9]:
id_para_linha['L194']

'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.'

In [10]:
id_para_linha['L195']

"Well, I thought we'd start with pronunciation, if that's okay with you."

#### Criação das listas com todas as conversas

In [11]:
conversas_id = []
for conversa in conversas[:-1]:
    _conversa = conversa.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversas_id.append(_conversa.split(','))

In [12]:
conversas_id[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

#### Separação das perguntas e respostas

In [13]:
perguntas = []
respostas = []
for conversa in conversas_id:
    #print(conversa)
    #print('****')
    for i in range(len(conversa) - 1):
        perguntas.append(id_para_linha[conversa[i]])
        respostas.append(id_para_linha[conversa[i + 1]])

In [14]:
perguntas[:10]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...']

In [15]:
respostas[:10]

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.']

In [16]:
len(perguntas), len(respostas)

(221616, 221616)

#### Teste com tradutor

In [17]:
#to_translate = ' '.join(perguntas[:100])
#perguntas_transleted = GoogleTranslator(source='english', target='portuguese').translate(to_translate)
#perguntas_transleted

In [18]:
#to_translate = ' '.join(respostas[:100])
#respostas_transleted = GoogleTranslator(source='english', target='portuguese').translate(to_translate)
#respostas_transleted

### Limpeza de texto

In [19]:
def limpa_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"i'm", "i am", texto)
    texto = re.sub(r"you're", "you are", texto)
    texto = re.sub(r"aren't", "are not", texto)
    texto = re.sub(r"he's", "he is", texto)
    texto = re.sub(r"she's", "she is", texto)
    texto = re.sub(r"that's", "that is", texto)
    texto = re.sub(r"what's", "what is", texto)
    texto = re.sub(r"where's", "where is", texto)
    texto = re.sub(r"don't", "do not", texto)
    texto = re.sub(r"didn't", "did not", texto)
    texto = re.sub(r"doesn't", "does not", texto)
    texto = re.sub(r"who's", "who is", texto)
    texto = re.sub(r"isn't", "not is", texto)
    texto = re.sub(r"let's", "let is", texto)
    texto = re.sub(r"mom's", "mother", texto)
    texto = re.sub(r"\'ll", " will", texto)
    texto = re.sub(r"\'ve", " have", texto)
    texto = re.sub(r"\'re", " are", texto)
    texto = re.sub(r"\'d", " would", texto)
    texto = re.sub(r"won't", "will not", texto)
    texto = re.sub(r"can't", "cannot", texto)
    texto = re.sub(r"c'mon", "came on", texto)
    
    texto = re.sub(r"[-()\"#/@;:<>{}`+=~|.?,]", "", texto)
    return texto

In [20]:
perguntas_limpas = []
for pergunta in perguntas:
    perguntas_limpas.append(limpa_texto(pergunta))

In [21]:
respostas_limpas = []
for resposta in respostas:
    respostas_limpas.append(limpa_texto(resposta))

In [22]:
perguntas_limpas[:5]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 "no no it's my fault  we did not have a proper introduction "]

In [23]:
respostas_limpas[:5]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron']

### Criação de um dicionário que mapeia cada palavra e o numero de ocorrencias

In [24]:
palavras_contagem = {}
for pergunta in perguntas_limpas:
    for palavra in pergunta.split():
        if palavra not in palavras_contagem:
            palavras_contagem[palavra] = 1
        else:
            palavras_contagem[palavra] += 1 
            
for resposta in respostas_limpas:
    for palavra in resposta.split():
        if palavra not in palavras_contagem:
            palavras_contagem[palavra] = 1
        else:
            palavras_contagem[palavra] += 1             

In [25]:
palavras_contagem

{'can': 15975,
 'we': 40500,
 'make': 6747,
 'this': 33573,
 'quick': 337,
 'roxanne': 1,
 'korrine': 1,
 'and': 65581,
 'andrew': 56,
 'barrett': 19,
 'are': 56024,
 'having': 1217,
 'an': 9482,
 'incredibly': 60,
 'horrendous': 4,
 'public': 364,
 'break': 895,
 'up': 16050,
 'on': 27769,
 'the': 140638,
 'quad': 2,
 'again': 3193,
 'well': 14090,
 'i': 204493,
 'thought': 4550,
 'would': 20005,
 'start': 1656,
 'with': 24954,
 'pronunciation': 2,
 'if': 18949,
 'that': 67068,
 'is': 87840,
 'okay': 6096,
 'you': 212914,
 'not': 84900,
 'hacking': 18,
 'gagging': 9,
 'spitting': 16,
 'part': 1419,
 'please': 3208,
 'asking': 746,
 'me': 44888,
 'out': 18466,
 'so': 19059,
 'cute': 272,
 'what': 55197,
 'your': 29938,
 'name': 3122,
 'no': 27572,
 "it's": 25843,
 'my': 29683,
 'fault': 482,
 'did': 21430,
 'have': 46591,
 'a': 101997,
 'proper': 138,
 'introduction': 19,
 'cameron': 35,
 'thing': 5731,
 'am': 37860,
 'at': 15290,
 'mercy': 68,
 'of': 56294,
 'particularly': 111,
 'hid

### Remoção de palavras não frequentes e tokenização

In [36]:
limite = 20
perguntas_palavras_in = {}
numero_palavras = 0
for palavra, contagem in palavras_contagem.items():
    if contagem >= limite:
        perguntas_palavras_in[palavra] = numero_palavras
        numero_palavras += 1
        
respostas_palavras_in = {}
numero_palavras = 0
for palavra, contagem in palavras_contagem.items():
    if contagem >= limite:
        respostas_palavras_in[palavra] = numero_palavras
        numero_palavras += 1 

In [29]:
perguntas_palavras_in

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'are': 7,
 'having': 8,
 'an': 9,
 'incredibly': 10,
 'public': 11,
 'break': 12,
 'up': 13,
 'on': 14,
 'the': 15,
 'again': 16,
 'well': 17,
 'i': 18,
 'thought': 19,
 'would': 20,
 'start': 21,
 'with': 22,
 'if': 23,
 'that': 24,
 'is': 25,
 'okay': 26,
 'you': 27,
 'not': 28,
 'part': 29,
 'please': 30,
 'asking': 31,
 'me': 32,
 'out': 33,
 'so': 34,
 'cute': 35,
 'what': 36,
 'your': 37,
 'name': 38,
 'no': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 'did': 43,
 'have': 44,
 'a': 45,
 'proper': 46,
 'cameron': 47,
 'thing': 48,
 'am': 49,
 'at': 50,
 'mercy': 51,
 'of': 52,
 'particularly': 53,
 'breed': 54,
 'loser': 55,
 'sister': 56,
 'cannot': 57,
 'date': 58,
 'until': 59,
 'she': 60,
 'does': 61,
 'why': 62,
 'mystery': 63,
 'used': 64,
 'to': 65,
 'be': 66,
 'really': 67,
 'popular': 68,
 'when': 69,
 'started': 70,
 'high': 71,
 'school': 72,
 'then': 73,
 'it': 74,
 'was': 75,
 'just': 76

In [30]:
len(perguntas_palavras_in)

8817

In [37]:
respostas_palavras_in

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'are': 7,
 'having': 8,
 'an': 9,
 'incredibly': 10,
 'public': 11,
 'break': 12,
 'up': 13,
 'on': 14,
 'the': 15,
 'again': 16,
 'well': 17,
 'i': 18,
 'thought': 19,
 'would': 20,
 'start': 21,
 'with': 22,
 'if': 23,
 'that': 24,
 'is': 25,
 'okay': 26,
 'you': 27,
 'not': 28,
 'part': 29,
 'please': 30,
 'asking': 31,
 'me': 32,
 'out': 33,
 'so': 34,
 'cute': 35,
 'what': 36,
 'your': 37,
 'name': 38,
 'no': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 'did': 43,
 'have': 44,
 'a': 45,
 'proper': 46,
 'cameron': 47,
 'thing': 48,
 'am': 49,
 'at': 50,
 'mercy': 51,
 'of': 52,
 'particularly': 53,
 'breed': 54,
 'loser': 55,
 'sister': 56,
 'cannot': 57,
 'date': 58,
 'until': 59,
 'she': 60,
 'does': 61,
 'why': 62,
 'mystery': 63,
 'used': 64,
 'to': 65,
 'be': 66,
 'really': 67,
 'popular': 68,
 'when': 69,
 'started': 70,
 'high': 71,
 'school': 72,
 'then': 73,
 'it': 74,
 'was': 75,
 'just': 76

In [38]:
len(respostas_palavras_in)

8817

#### Adição de tokens no dicionários

In [39]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    perguntas_palavras_in[token] = len(perguntas_palavras_in) + 1

for token in tokens:
    respostas_palavras_in[token] = len(respostas_palavras_in) + 1    

In [40]:
len(respostas_palavras_in)

8821

In [41]:
len(perguntas_palavras_in)

8821

In [44]:
perguntas_palavras_in['<OUT>']

8820

In [42]:
respostas_palavras_in['<OUT>']

8820

In [45]:
respostas_palavras_in['bugger']

7446

#### Criação de dicionário inverso com dicionário respostas

In [48]:
respostas_int_palavras = {p_i: p for p, p_i in respostas_palavras_in.items()}

In [49]:
respostas_int_palavras

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'quick',
 5: 'and',
 6: 'andrew',
 7: 'are',
 8: 'having',
 9: 'an',
 10: 'incredibly',
 11: 'public',
 12: 'break',
 13: 'up',
 14: 'on',
 15: 'the',
 16: 'again',
 17: 'well',
 18: 'i',
 19: 'thought',
 20: 'would',
 21: 'start',
 22: 'with',
 23: 'if',
 24: 'that',
 25: 'is',
 26: 'okay',
 27: 'you',
 28: 'not',
 29: 'part',
 30: 'please',
 31: 'asking',
 32: 'me',
 33: 'out',
 34: 'so',
 35: 'cute',
 36: 'what',
 37: 'your',
 38: 'name',
 39: 'no',
 40: "it's",
 41: 'my',
 42: 'fault',
 43: 'did',
 44: 'have',
 45: 'a',
 46: 'proper',
 47: 'cameron',
 48: 'thing',
 49: 'am',
 50: 'at',
 51: 'mercy',
 52: 'of',
 53: 'particularly',
 54: 'breed',
 55: 'loser',
 56: 'sister',
 57: 'cannot',
 58: 'date',
 59: 'until',
 60: 'she',
 61: 'does',
 62: 'why',
 63: 'mystery',
 64: 'used',
 65: 'to',
 66: 'be',
 67: 'really',
 68: 'popular',
 69: 'when',
 70: 'started',
 71: 'high',
 72: 'school',
 73: 'then',
 74: 'it',
 75: 'was',
 76: 'just'

In [50]:
respostas_int_palavras[8000]

'fossils'

#### Adição do token final de string <EOS> para o final de cada resposta

In [55]:
for i in range(len(respostas_limpas)):
    respostas_limpas[i] += ' <EOS>'

In [56]:
respostas_limpas

['well i thought we would start with pronunciation if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part  please <EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>',
 'right  see  you are ready for the quiz <EOS>',
 'i do not want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone <EOS>',
 "that is because it's such a nice one <EOS>",
 'forget french <EOS>',
 "we

#### Tradução de todas as perguntas e respostas para inteiros

In [59]:
perguntas_para_int = []
for pergunta in perguntas_limpas:
    ints = []
    for palavra in pergunta.split():
        if palavra not in perguntas_palavras_in:
            ints.append(perguntas_palavras_in['<OUT>'])
        else:
            ints.append(perguntas_palavras_in[palavra])
    perguntas_para_int.append(ints)

In [60]:
perguntas_para_int

[[0,
  1,
  2,
  3,
  4,
  8820,
  8820,
  5,
  6,
  8820,
  7,
  8,
  9,
  10,
  8820,
  11,
  12,
  13,
  14,
  15,
  8820,
  16],
 [17, 18, 19, 1, 20, 21, 22, 8820, 23, 24, 25, 26, 22, 27],
 [28, 15, 8820, 5, 8820, 5, 8820, 29, 30],
 [27, 7, 31, 32, 33, 24, 25, 34, 35, 36, 25, 37, 38, 16],
 [39, 39, 40, 41, 42, 1, 43, 28, 44, 45, 46, 8820],
 [47],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8820,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61],
 [62],
 [8820,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81],
 [82, 23, 83, 1, 84, 85, 86, 45, 87],
 [8820, 88, 8820, 3, 25, 41, 89],
 [90, 91, 27, 7, 92, 93, 15, 8820],
 [18,
  94,
  28,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112

In [61]:
respostas_para_int = []
for resposta in respostas_limpas:
    ints = []
    for palavra in resposta.split():
        if palavra not in respostas_palavras_in:
            ints.append(respostas_palavras_in['<OUT>'])
        else:
            ints.append(respostas_palavras_in[palavra])
    respostas_para_int.append(ints)

In [62]:
respostas_para_int

[[17, 18, 19, 1, 20, 21, 22, 8820, 23, 24, 25, 26, 22, 27, 8819],
 [28, 15, 8820, 5, 8820, 5, 8820, 29, 30, 8819],
 [26, 73, 97, 1524, 1, 853, 33, 478, 383, 8820, 208, 240, 8819],
 [243, 74, 8819],
 [47, 8819],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8820,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61,
  8819],
 [395, 77, 60, 84, 128, 45, 58, 858, 287, 8819],
 [8820,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81,
  8819],
 [24, 25, 45, 1692, 8819],
 [285, 32, 91, 36, 18, 0, 94, 8819],
 [90, 91, 27, 7, 92, 93, 15, 8820, 8819],
 [18,
  94,
  28,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112,
  113,
  65,
  114,
  33,
  41,
  89,
  65,
  115,
  8819],
 [24, 25, 116, 40, 117, 

#### Ordenação das perguntas e respostas pelo tamanho das perguntas

In [67]:
perguntas_limpas_ordenadas = []
respostas_limpas_ordenadas = []
for tamanho in range(1, 26):
    for i in enumerate(perguntas_para_int):
        if len(i[1]) == tamanho:
            perguntas_limpas_ordenadas.append(perguntas_para_int[i[0]])
            respostas_limpas_ordenadas.append(respostas_para_int[i[0]])

In [70]:
len(perguntas_limpas_ordenadas)

203513

In [71]:
len(respostas_limpas_ordenadas)

203513

In [72]:
perguntas_limpas_ordenadas

[[47],
 [62],
 [123],
 [146],
 [134],
 [39],
 [174],
 [39],
 [180],
 [181],
 [219],
 [36],
 [62],
 [134],
 [62],
 [110],
 [371],
 [131],
 [295],
 [209],
 [36],
 [220],
 [180],
 [26],
 [62],
 [180],
 [8820],
 [451],
 [248],
 [180],
 [191],
 [8820],
 [8820],
 [662],
 [97],
 [8820],
 [8820],
 [36],
 [39],
 [36],
 [8820],
 [146],
 [90],
 [36],
 [764],
 [8820],
 [631],
 [39],
 [39],
 [932],
 [8820],
 [1113],
 [39],
 [229],
 [69],
 [209],
 [39],
 [141],
 [1259],
 [209],
 [1105],
 [1105],
 [1105],
 [1105],
 [209],
 [337],
 [148],
 [26],
 [92],
 [662],
 [36],
 [1132],
 [1254],
 [8820],
 [8820],
 [1502],
 [209],
 [1543],
 [36],
 [36],
 [209],
 [1592],
 [1592],
 [1592],
 [1592],
 [1592],
 [1592],
 [26],
 [662],
 [67],
 [662],
 [229],
 [97],
 [1621],
 [1592],
 [1592],
 [1592],
 [8820],
 [1592],
 [1592],
 [67],
 [1662],
 [667],
 [1782],
 [209],
 [38],
 [209],
 [1216],
 [209],
 [1342],
 [209],
 [17],
 [123],
 [36],
 [62],
 [1831],
 [1764],
 [209],
 [1839],
 [209],
 [209],
 [1814],
 [220],
 [1140],


In [73]:
respostas_limpas_ordenadas

[[15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8820,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61,
  8819],
 [8820,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81,
  8819],
 [102, 8819],
 [1520, 77, 101, 1541, 33, 148, 602, 8819],
 [27, 152, 225, 3, 6440, 8819],
 [26, 27, 7, 159, 251, 65, 1272, 97, 65, 607, 8819],
 [285, 25, 133, 8819],
 [27, 237, 132, 192, 224, 74, 8819],
 [194, 8819],
 [20, 27, 124, 606, 32, 45, 1503, 47, 8819],
 [41,
  2572,
  156,
  18,
  44,
  78,
  45,
  103,
  1104,
  50,
  956,
  15,
  8820,
  143,
  217,
  513,
  8819],
 [111, 8820, 93, 45, 269, 8819],
 [178, 75, 77, 45, 270, 271, 8819],
 [277,
  24,
  18,
  278,
  18,
  20,
  110,
  94,
  279,
  76,
  116,
  276,
  265,
  75,
  262,
  74,
  5,
  18,
  280,
  281,
  282,
  93,
  8820,
  151,
  5,
  41,
  8820,
  8820,
  283,
  8819],
 [116, 60, 242, 4614, 379, 295, 8819

#### Construção do modelo adaptado para a ultima versão do tensorflow

In [74]:
print(tf.__version__)

2.12.0


## Construção do modelo Seq2Seq

#### Criação de placeholders para as entradas e saidas

In [75]:
def entradas_modelo():
    ## Criação dos placeholders
    entradas = tf.compat.v1.placeholder(tf.int32, shape=(None, None), name = 'entradas')
    saidas = tf.compat.v1.placeholder(tf.int32, shape=(None, None), name = 'saidas')
    lr = tf.compat.v1.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name = 'keep_prob')
    
    return entradas, saidas, lr, keep_prob

In [83]:
respostas_int_palavras[8234]

'panther'

#### Pré Processamento das saidas (alvos)

In [80]:
def preprocessamento_saidas(saidas, palavra_para_int, bash_size):
    esquerda = tf.fill([bash_size, 1], palavra_para_int['<SOS>'])
    direita = tf.strided_slice(saidas, [0,0], [bash_size, -1], strides=[1,1])
    saidas_preprocessadas = tf.concat([esquerda, direita], 1)
    return saidas_preprocessadas

In [81]:
respostas_palavras_in['<SOS>']

8821

#### Criacação da camada RNN do codificador

In [84]:
def rnn_codificador(rnn_entradas, rnn_tamanho, numero_camadas, keep_prob, tamanho_sequencia):
    lstm = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_tamanho)
    lstm_dropout = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm, input_keep_prob=keep_prob)
    encoder_celula = tf.compat.v1.nn.rnn_cell.MultiRNNCell([lstm_dropout] * numero_camadas)
    encoder_estado = tf.compat.v1.nn.bidirectional_dynamic_rnn(cell_fw=encoder_celula, cell_bw=encoder_celula,
                                                              sequence_length=tamanho_sequencia,
                                                              inputs=rnn_entradas,
                                                              dtype=tf.float32)
    return encoder_estado